In [1]:
import pandas as pd
from wKit.ML.sk_ml import grid_cv_a_model, grid_cv_default_params
import warnings
from sklearn.exceptions import UndefinedMetricWarning
warnings.filterwarnings("ignore", category=UndefinedMetricWarning)
from sklearn.metrics import f1_score

import xgboost

In [2]:
def load_data():
    Xs = {'RoadNet': pd.read_csv('data/x_RoadNet.csv', index_col=0)}

    for ftr_type in ['Segment', 'RoadNet+Segment']:
        for total_or_not in ['NO_TOTAL', 'TOTAL']:
            feature_combo_name = '%s_%s_%s' % (ftr_type, total_or_not, '~2014')
            Xs[feature_combo_name] = pd.read_csv('data/x_%s.csv' % feature_combo_name, index_col=0)

    return Xs

In [3]:
Xs = load_data()

In [4]:
y = pd.read_csv('data/y_csl_all-2017-10-01.csv', index_col=0).csl.round()

In [5]:
X = Xs['RoadNet+Segment_NO_TOTAL_~2014']
X = X.loc[y.index]

In [7]:
idx_list = y.index.tolist()
pd.np.random.shuffle(idx_list)

df_all = []
f1_micro = []
f1_macro = []
for idx in pd.np.array_split(idx_list,5):
    test_y = y.loc[idx]
    train_y = y[~y.index.isin(idx)]
    test_X = X.loc[idx]
    train_X = X[~X.index.isin(idx)]
    cls = xgboost.XGBClassifier()
    params = grid_cv_default_params()['cls']['XGBcls']
    res = grid_cv_a_model(train_X, train_y, cls, params, 'cls', 'XGBcls', save_res=False)    
    best_cls = res['best_model']
    pred = best_cls.predict(test_X)
    f1_micro.append(f1_score(test_y, pred, average='micro'))
    f1_macro.append(f1_score(test_y, pred, average='macro'))
    df_true_pred = test_y.to_frame()
    df_true_pred['pred'] = pred
    df_all.append(df_true_pred)
df_all=pd.concat(df_all)

2017-10-21 10:36:05.917203 CVing: kind = cls, model = XGBcls
Fitting 5 folds for each of 64 candidates, totalling 320 fits


[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:   40.1s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:  2.9min
[Parallel(n_jobs=4)]: Done 320 out of 320 | elapsed:  5.3min finished


2017-10-21 10:41:29.966753 CVing: kind = cls, model = XGBcls
Fitting 5 folds for each of 64 candidates, totalling 320 fits


[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:   46.1s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:  3.2min
[Parallel(n_jobs=4)]: Done 320 out of 320 | elapsed:  5.5min finished


2017-10-21 10:47:06.125333 CVing: kind = cls, model = XGBcls
Fitting 5 folds for each of 64 candidates, totalling 320 fits


/home/jhwu92/miniconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:597: Warning: The least populated class in y has only 3 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:   41.3s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:  2.9min
[Parallel(n_jobs=4)]: Done 320 out of 320 | elapsed:  5.0min finished


2017-10-21 10:52:10.964233 CVing: kind = cls, model = XGBcls
Fitting 5 folds for each of 64 candidates, totalling 320 fits


[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:   39.6s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:  2.8min
[Parallel(n_jobs=4)]: Done 320 out of 320 | elapsed:  4.9min finished


2017-10-21 10:57:10.145836 CVing: kind = cls, model = XGBcls
Fitting 5 folds for each of 64 candidates, totalling 320 fits


[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:   39.7s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:  2.8min
[Parallel(n_jobs=4)]: Done 320 out of 320 | elapsed:  5.0min finished


In [14]:
df_all.to_csv('poster used true and pred.csv')

In [11]:
f1_score(df_all.csl, df_all.pred, average='micro')

0.61990950226244346

In [12]:
import geopandas as gpd

ModuleNotFoundError: No module named 'geopandas'